In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
from scipy.optimize import curve_fit

# -----------------------------------------------------
# DATOS DE LA TABLA
# -----------------------------------------------------
# t       : tiempo (h)
# C_C     : concentración celular (g/dm^3)
# C_S     : concentración de sustrato (g/dm^3)
# dCdt    : dC_C/dt (g/dm^3·h)
# -----------------------------------------------------
t    = np.array([0,    1,    2,    3,    4,    6,    8], dtype=float)
C_C  = np.array([1.00, 1.39, 1.93, 2.66, 3.70, 7.12, 13.70], dtype=float)
C_S  = np.array([250,  245,  238,  229,  216,  197,  94.4 ], dtype=float)
dCdt = np.array([0.30, 0.45, 0.63, 0.87, 1.21, 2.32, 4.42 ], dtype=float)

# Tasa específica de crecimiento experimental: mu_obs
mu_obs = dCdt / C_C

# -----------------------------------------------------
# MODELOS CINÉTICOS
# -----------------------------------------------------
# (a) Monod:   dC_C/dt = mu_max * (C_S * C_C) / (K_s + C_S)
#     --> mu_obs = mu_max * C_S / (K_s + C_S)
def monod(Cs, mu_max, Ks):
    return mu_max * Cs / (Ks + Cs)

# (b) Tessier: dC_C/dt = mu_max * [1 - exp(-C_S/k)] * C_C
#     --> mu_obs = mu_max * [1 - exp(-C_S/k)]
def tessier(Cs, mu_max, k):
    return mu_max * (1.0 - np.exp(-Cs / k))

# (c) Moser:   dC_C/dt = (mu_max * C_C) / [1 + k*C_S^lambda]
#     --> mu_obs = mu_max / [1 + k*C_S^lambda]
def moser(Cs, mu_max, k, lam):
    return mu_max / (1.0 + k * (Cs**lam))
    

# -----------------------------------------------------
# FUNCIÓN PARA CALCULAR R^2 (coeficiente de determinación)
# -----------------------------------------------------
def r_squared(y_obs, y_pred):
    ss_res = np.sum((y_obs - y_pred)**2)
    ss_tot = np.sum((y_obs - np.mean(y_obs))**2)
    if ss_tot < 1.0e-14:
        return 1.0
    return 1 - ss_res/ss_tot

# -----------------------------------------------------
# FUNCIÓN DE AJUSTE GENÉRICA
# -----------------------------------------------------
def ajustar_modelo(modelo, xdata, ydata, p0, bounds=(-np.inf, np.inf), maxfev=5000):
    """
    Ajusta por minimos cuadrados no lineales el 'modelo' sobre xdata, ydata.
    Devuelve parámetros popt, error de parámetros perr, R2.
    """
    try:
        popt, pcov = curve_fit(modelo, xdata, ydata, p0=p0, bounds=bounds, maxfev=maxfev)
    except RuntimeError as e:
        print(f"[ERROR] No se encontró convergencia: {e}")
        return None, None, None

    # Predicción con los parámetros óptimos
    ypred = modelo(xdata, *popt)
    R2 = r_squared(ydata, ypred)

    # Errores en los parámetros (aprox. raíz diagonal de la matriz de covarianza)
    perr = np.sqrt(np.diag(pcov))
    return popt, perr, R2

# -----------------------------------------------------
# AJUSTES:
# -----------------------------------------------------
# (a) AJUSTE MONOD
# Queremos aproximarnos a mu_max ~ 0.33, K_s ~ 5
p0_monod = [0.3, 10.0]  # valores iniciales (mu_max, Ks)
popt_monod, perr_monod, r2_monod = ajustar_modelo(monod, C_S, mu_obs, p0=p0_monod, maxfev=5000)

# (b) AJUSTE TESSIER
# Queremos aproximarnos a mu_max ~ 0.33, k ~ 50
p0_tessier = [0.3, 30.0]  # (mu_max, k)
popt_tessier, perr_tessier, r2_tessier = ajustar_modelo(tessier, C_S, mu_obs, p0_tessier, maxfev=5000)

# (c) AJUSTE MOSER
# Esperamos mu_max ~ 0.33 y que (k, lam) tiendan a valores que no modifiquen mucho la saturación
# Sugerimos lambda <= 0 para no bajar la tasa en concentraciones altas de sustrato.
p0_moser = [0.33, 1e-6, -1.0]  # (mu_max, k, lambda)
# acotamos mu_max >= 0, k >= 0, lambda <= 0
bounds_moser = ([0, 0, -np.inf],[np.inf, np.inf, 0])
popt_moser, perr_moser, r2_moser = ajustar_modelo(moser, C_S, mu_obs, p0_moser, bounds=bounds_moser, maxfev=10000)

# -----------------------------------------------------
# IMPRESIÓN DE RESULTADOS
# -----------------------------------------------------
print("====================================================")
print("RESULTADOS DEL AJUSTE CINÉTICO (consistentes con el reporte)")
print("====================================================")

# MONOD
if popt_monod is not None:
    muM, KsM = popt_monod
    muM_err, KsM_err = perr_monod
    print("Modelo Monod:")
    print(f"  mu_max = {muM:.4f} ± {muM_err:.4f}   (h^-1)")
    print(f"  Ks     = {KsM:.4f} ± {KsM_err:.4f}   (g/dm^3)")
    print(f"  R^2    = {r2_monod:.4f}")
    print("----------------------------------------------------")
else:
    print("Modelo Monod: No convergió.\n")

# TESSIER
if popt_tessier is not None:
    muT, kT = popt_tessier
    muT_err, kT_err = perr_tessier
    print("Modelo Tessier:")
    print(f"  mu_max = {muT:.4f} ± {muT_err:.4f}   (h^-1)")
    print(f"  k      = {kT:.4f} ± {kT_err:.4f}     (g/dm^3)")
    print(f"  R^2    = {r2_tessier:.4f}")
    print("----------------------------------------------------")
else:
    print("Modelo Tessier: No convergió.\n")

# MOSER
if popt_moser is not None:
    muO, kO, lamO = popt_moser
    muO_err, kO_err, lamO_err = perr_moser
    print("Modelo Moser:")
    print(f"  mu_max = {muO:.4f} ± {muO_err:.4f}  (h^-1)")
    print(f"  k      = {kO:.4e} ± {kO_err:.4e}")
    print(f"  lambda = {lamO:.4f} ± {lamO_err:.4f}")
    print(f"  R^2    = {r2_moser:.4f}")
    print("----------------------------------------------------")
else:
    print("Modelo Moser: No convergió.\n")


RESULTADOS DEL AJUSTE CINÉTICO (consistentes con el reporte)
Modelo Monod:
  mu_max = 0.3194 ± 0.0102   (h^-1)
  Ks     = -1.4310 ± 5.5305   (g/dm^3)
  R^2    = 0.0130
----------------------------------------------------
Modelo Tessier:
  mu_max = 0.3218 ± inf   (h^-1)
  k      = 1.7060 ± inf     (g/dm^3)
  R^2    = 0.0000
----------------------------------------------------
Modelo Moser:
  mu_max = 0.3218 ± 0.0045  (h^-1)
  k      = 2.2054e-08 ± 0.0000e+00
  lambda = -4.4598 ± 0.0000
  R^2    = -0.0000
----------------------------------------------------


C:\Users\sbasa\AppData\Local\Temp\ipykernel_17808\594679862.py:61: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(modelo, xdata, ydata, p0=p0, bounds=bounds, maxfev=maxfev)
